In [1]:
import pandas as pd

In [23]:
df = pd.read_csv('2024_LoL_esports_match_data_from_OraclesElixir.csv')

In [24]:
df.columns

Index(['gameid', 'datacompleteness', 'url', 'league', 'year', 'split',
       'playoffs', 'date', 'game', 'patch',
       ...
       'opp_csat15', 'golddiffat15', 'xpdiffat15', 'csdiffat15', 'killsat15',
       'assistsat15', 'deathsat15', 'opp_killsat15', 'opp_assistsat15',
       'opp_deathsat15'],
      dtype='object', length=131)

In [6]:
df.query('datacompleteness == "partial"').count()

gameid              15564
datacompleteness    15564
url                 15564
league              15564
year                15564
                    ...  
assistsat15             0
deathsat15              0
opp_killsat15           0
opp_assistsat15         0
opp_deathsat15          0
Length: 131, dtype: int64

In [7]:
df.query('datacompleteness == "complete"').count()

gameid              69852
datacompleteness    69852
url                     0
league              69852
year                69852
                    ...  
assistsat15         69840
deathsat15          69840
opp_killsat15       69840
opp_assistsat15     69840
opp_deathsat15      69840
Length: 131, dtype: int64

In [25]:
df2 = pd.read_csv('2021_LoL_esports_match_data_from_OraclesElixir.csv')

In [26]:
df2.columns

Index(['gameid', 'datacompleteness', 'url', 'league', 'year', 'split',
       'playoffs', 'date', 'game', 'patch',
       ...
       'opp_csat15', 'golddiffat15', 'xpdiffat15', 'csdiffat15', 'killsat15',
       'assistsat15', 'deathsat15', 'opp_killsat15', 'opp_assistsat15',
       'opp_deathsat15'],
      dtype='object', length=131)

In [9]:
df.query('datacompleteness == "partial"').count()

gameid              13008
datacompleteness    13008
url                 13008
league              13008
year                13008
                    ...  
assistsat15           108
deathsat15            108
opp_killsat15         108
opp_assistsat15       108
opp_deathsat15        108
Length: 131, dtype: int64

In [16]:
columns = [e for e in df.query('datacompleteness == "complete"').columns if e.endswith('10')]

In [17]:
columns

['goldat10',
 'xpat10',
 'csat10',
 'opp_goldat10',
 'opp_xpat10',
 'opp_csat10',
 'golddiffat10',
 'xpdiffat10',
 'csdiffat10',
 'killsat10',
 'assistsat10',
 'deathsat10',
 'opp_killsat10',
 'opp_assistsat10',
 'opp_deathsat10']

In [22]:
from pprint import pprint
pprint(list(e for e in df.query('datacompleteness == "complete"').columns))

['gameid',
 'datacompleteness',
 'url',
 'league',
 'year',
 'split',
 'playoffs',
 'date',
 'game',
 'patch',
 'participantid',
 'side',
 'position',
 'playername',
 'playerid',
 'teamname',
 'teamid',
 'champion',
 'ban1',
 'ban2',
 'ban3',
 'ban4',
 'ban5',
 'pick1',
 'pick2',
 'pick3',
 'pick4',
 'pick5',
 'gamelength',
 'result',
 'kills',
 'deaths',
 'assists',
 'teamkills',
 'teamdeaths',
 'doublekills',
 'triplekills',
 'quadrakills',
 'pentakills',
 'firstblood',
 'firstbloodkill',
 'firstbloodassist',
 'firstbloodvictim',
 'team kpm',
 'ckpm',
 'firstdragon',
 'dragons',
 'opp_dragons',
 'elementaldrakes',
 'opp_elementaldrakes',
 'infernals',
 'mountains',
 'clouds',
 'oceans',
 'chemtechs',
 'hextechs',
 'dragons (type unknown)',
 'elders',
 'opp_elders',
 'firstherald',
 'heralds',
 'opp_heralds',
 'void_grubs',
 'opp_void_grubs',
 'firstbaron',
 'barons',
 'opp_barons',
 'firsttower',
 'towers',
 'opp_towers',
 'firstmidtower',
 'firsttothreetowers',
 'turretplates',
 'op

In [28]:
!pip install pulsefire pulsefire-types -U
!pip install --upgrade certifi


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:
from pulsefire.clients import RiotAPIClient

In [156]:
from pulsefire.schemas import *
from pprint import pprint

In [215]:
import os
API_KEY = os.getenv('RIOT_API_KEY')

async def get_division(tier='DIAMOND', division='I', queue='RANKED_SOLO_5x5'):
    async with RiotAPIClient(default_headers={"X-Riot-Token": API_KEY}) as client:
        return await client.get_lol_league_v4_entries_by_division(tier=tier, division=division, queue=queue, region='euw1')

async def get_player_info(summonerId):
    async with RiotAPIClient(default_headers={"X-Riot-Token": API_KEY}) as client:
        return await client.get_lol_summoner_v4_by_id(id=summonerId, region='euw1')

async def get_match_history(summoner_puuid):
    async with RiotAPIClient(default_headers={"X-Riot-Token": API_KEY}) as client:
        return await client.get_lol_match_v5_match_ids_by_puuid(puuid=summoner_puuid, region='europe')

async def get_match_timeline(match_id='EUW1_7044136381'):
    async with RiotAPIClient(default_headers={"X-Riot-Token": API_KEY}) as client:
        return await client.get_lol_match_v5_match_timeline(id=match_id, region='europe')

async def get_match(match_id):
    async with RiotAPIClient(default_headers={"X-Riot-Token": API_KEY}) as client:
        return await client.get_lol_match_v5_match(id=match_id, region='europe')

In [14]:
import pandas as pd
import json

RELEVANT_COLUMNS = ('side', 'jungleMinionsKilled', 'level', 'minionsKilled', 'totalGold', 'xp', 'totalDamageDoneToChampions', 'totalDamageTaken')

def extract_player_match_data(match, frame=15):
    df = pd.DataFrame.from_dict(match['info']['frames'][frame]['participantFrames'], orient='index')
    df = df.drop(columns=['damageStats']).reset_index().join(pd.json_normalize(df['damageStats'])).set_index('index')
    #df = df.join(pd.json_normalize(df['damageStats']))
    df['side'] = ['blue' if int(idx) > 5 else 'red' for idx in df.index]
    return df[[*RELEVANT_COLUMNS]]


In [162]:
division_players = await get_division()


[{'leagueId': 'a7fc3952-7354-40d9-80ca-ca14ac15687b',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'DIAMOND',
  'rank': 'I',
  'summonerId': 'BfguOcsFT20IsvJ5iyxouTn9PCffkxPh2fUE7zyac3U-tsAT',
  'leaguePoints': 22,
  'wins': 20,
  'losses': 10,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False},
 {'leagueId': 'cae4cbaa-692a-4aa4-8958-bbc5d1fe049f',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'DIAMOND',
  'rank': 'I',
  'summonerId': '19Z8MsZr7wsESRBIe9By7CEVdq6RBFVqENZVoYuw3hoY765S',
  'leaguePoints': 59,
  'wins': 135,
  'losses': 144,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False},
 {'leagueId': '6e6b7125-002a-430a-88e4-6351ecb3389a',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'DIAMOND',
  'rank': 'I',
  'summonerId': 'gfaiSD9ah4dZmmQaa0rEo3H9yfw1xYHBeBTncRQcZ6pgY8z4',
  'leaguePoints': 75,
  'wins': 28,
  'losses': 23,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False},
 {'le

In [164]:
example_player_id = division_players[0]['summonerId']

In [169]:
player_info = await get_player_info(example_player_id)

In [171]:
example_puuid = player_info['puuid']

In [177]:
match_history = await get_match_history(example_puuid)

In [179]:
match_history[0]

'EUW1_7049600439'

In [180]:
timeline = await get_match_timeline(match_history[0])

In [228]:
match = await get_match(match_history[1])


In [229]:
match['info']['participants'][4]['lane']

'TOP'

In [218]:
timeline

{'metadata': {'dataVersion': '2',
  'matchId': 'EUW1_7049600439',
  'participants': ['budiTqpOJU-wLUqQ_NhqL9HOBBmoQ3CAXr1gOCLECMoqtp9__TRCykJGKPf1womgoknE5yGhW_NOeg',
   '3PNlxzjDQirdI6h3-t4FkMjZtFbP7qsfQP2IUrX-T9Gc87wuGeJ17o9mud70a0nykrijU9gWvWGy5w',
   'nJwl-sPl_BI92dx56G6SsVi5AoKJdgY6nr19_HgBf9odzoNDlXnMmrfMosEABxpOFHECyH1pH1ZGuw',
   'SnMa9Bg3tEzzAm7hNWUNyCMbZ-7-87GPm0pV9xQum3HGwUCBTm8gz8AR1keYnUP84Z3iGHlcINujQg',
   '3IIeCRPm_sY7MJMfyoxTWL18WL9v_rtFNKDkPmwsb6LMn6xOqFRNCqQXItlxX0wI7gZq4w0kzIVViA',
   'H7iNx6prMyTnuQIvNIfVnqM6nPPwv-MrUnCjcQuOyRFqxeazcbPK4RVytzgJA8UzqOUAkU_Gz26sog',
   'x9-4xNbQC7QkMeng92cWWpgVi3uwy64AATGvZoV-VHWRdTFafjv5ASYOu7bNmLZHT6jzdi96XWBfPg',
   '_UnAB__jV4V62LyJTS-jAMfu9hHBEelfrxqu0i1POlrRI3E9VFe6iYAkCabKoHfEdYFL1kJnE2tIAg',
   'V_lhDVS5gtdxprV2Dfq-K1AA9NYleEqkpA01QRyhfYoxZA-c5sepPOtgz0DGd5b54woEW8ng5EqTyA',
   'pxByJi2C88HPU8K7SAaPil3VkkaVrszj-Xo5UaqguRrUJwObD9RY4n9sFVLt16ouIzW4kMteEGe6Cg']},
 'info': {'endOfGameResult': 'GameComplete',
  'frameInterval': 60

In [13]:
data = extract_match_data(timeline, frame=15)

NameError: name 'extract_match_data' is not defined

In [12]:
data

{'metadata': {'dataVersion': '2',
  'matchId': 'EUW1_6974140969',
  'participants': ['B847iTKbKQfV0S3bwE81qMnOxspVt7-pvMd7kRrTm_ScIyUsljAYc5ewzBV2WcvAKnIB24VXiOx8sg',
   'DeNQ9vLK2tsPCiA_hoTIVQcSZAb9L4ATWD_s44rSVIREsEx4wXiYJvJJ0VRb4JhSrL5xi7Xbu20ecQ',
   'IhsAYGwvaincs27oBU_ITd9dtWlhVvQmiFbLtDdAPqsUWqs9T4XW3V8PKOVJ2B56kjIDudNV6cR25w',
   'p_u1SlWPYH3HjfHzzmMiOta8LSgk07_eKlP8ZzVmWfGh1YKwq3QdV44zTk6t-u3Mv_TzDyUGR9T0Sw',
   'cIGOGTNN_p4xv3U6ROCL6-uFXiHk73TZOrplZAP9TX6SxJ0457Dq8XLInoSxp-5hDduByyOW-Y9s6w',
   'oWNDSsGn1x-qoG8rvBLCiZsbLxNRxbvcKaL1lEeCszOI4PwZ3ILenLbxPB00NIdgUpUwERSyC9ePGg',
   'Gqz62LjuOLTqbLKaKPvtg9bkAfPJoATWJxUPqpDUyumAE91HGDNGSTgLHLsesNoXYXhLqusXu9nzzw',
   'crnPpeIviImzIahWEz8CGvInljzFg_9HLGVh_FhIv5zIQdY6IflqYy9SNs5w68dzrDehxfXNT9Pr3A',
   'ukOGkJHgquRh3Hp00UKiyLQ42Ejc_Nv6t0dyocfDer9298vDrp1ZaT47rbGEI0hg2ghl_sEZAzPUVA',
   'FNaWr0GR5_0nrCkCVTJwjCYjTM_FQaZ7FyoQelhKwIilayhOUAcu274kHR9GW3aPd16p0KH68T-eZw']},
 'info': {'endOfGameResult': 'GameComplete',
  'gameCreation': 171

In [148]:
extracted_data

#Los 5 primeros son red side

# 1 - Camille - TOP
# 2 - Kaisa   - ADC
# 3 - Lux     - SUP
# 4 - Rammus  - JNG
# 5 - Zilean  - MID

# 6 - Yuumi
# 7 - 
# 8 - Briar 
# 9 -
# 10 -

,side,jungleMinionsKilled,level,minionsKilled,totalGold,xp,totalDamageDoneToChampions,totalDamageTaken
index,,,,,,,,
1,red,0,14,142,8868,12978,22098,29080
2,red,4,13,149,9165,11133,15167,22386
3,red,0,12,39,8042,9466,6917,14851
4,red,114,14,31,8897,11447,10838,35227
5,red,0,16,163,8608,14758,6612,22291
6,blue,0,15,10,8773,13457,10512,2500
7,blue,8,14,182,10327,12156,14811,37900
8,blue,153,14,16,11902,12494,18630,30217
9,blue,23,15,253,16252,14178,38141,13768


In [213]:
def process_frames(timeline):
    parsed_events = []
    for frame in timeline['info']['frames']:
        events = frame['events']
        for ev in events:
            if ev['type'] not in ('CHAMPION_KILL', 'ELITE_MONSTER_KILL', 'BUILDING_KILL', 'CHAMPION_SPECIAL_KILL'):
                continue
                
            match ev['type']:
                case 'CHAMPION_KILL':
                    data = {'assists': ev.get('assistingParticipantIds', []), 'killer': ev['killerId']}
                case 'ELITE_MONSTER_KILL':
                    data = {'monster': ev.get('monsterType'), 'team_id': ev['killerTeamId']}
                case 'BUILDING_KILL':
                    data = {'building': ' '.join([ev['buildingType'], ev.get('towerType', '')]), 'lane': ev['laneType'], 'team_id': ev['teamId']}
            parsed_events.append(data)
            
    return parsed_events
        

In [214]:
process_frames(timeline)

[{'assists': [9], 'killer': 10},
 {'assists': [9], 'killer': 10},
 {'assists': [2], 'killer': 5},
 {'assists': [1], 'killer': 4},
 {'assists': [], 'killer': 2},
 {'assists': [], 'killer': 10},
 {'assists': [], 'killer': 5},
 {'assists': [10], 'killer': 7},
 {'assists': [1, 3], 'killer': 4},
 {'assists': [1], 'killer': 4},
 {'assists': [5], 'killer': 2},
 {'monster': 'HORDE', 'team_id': 100},
 {'monster': 'HORDE', 'team_id': 100},
 {'monster': 'HORDE', 'team_id': 100},
 {'monster': 'DRAGON', 'team_id': 200},
 {'assists': [5], 'killer': 2},
 {'assists': [], 'killer': 10},
 {'monster': 'HORDE', 'team_id': 100},
 {'monster': 'HORDE', 'team_id': 100},
 {'monster': 'HORDE', 'team_id': 100},
 {'assists': [1, 4, 5], 'killer': 3},
 {'assists': [], 'killer': 4},
 {'assists': [1, 3], 'killer': 5},
 {'assists': [1], 'killer': 4},
 {'building': 'TOWER_BUILDING OUTER_TURRET',
  'lane': 'BOT_LANE',
  'team_id': 200},
 {'monster': 'DRAGON', 'team_id': 100},
 {'assists': [8], 'killer': 10},
 {'assists'

In [11]:
import os
import glob
import json
from collections import defaultdict

# Path to the folder containing the JSON files
folder_path = '/bronze/match/details/'

# Use glob to get all JSON files in the folder
json_files = glob.glob(os.path.join(folder_path, '*.json'))

# Dictionary to store the count of each game mode
game_mode_counts = defaultdict(int)
ranked_game_counts = defaultdict(int)

# Function to determine if the game is ranked or not
def is_ranked(game_mode):
    # Assuming that ranked modes are named explicitly or have some identifiable feature in the mode name
    ranked_modes = {'RANKED_SOLO_5x5', 'RANKED_FLEX_SR', 'RANKED_TFT', 'RANKED_TFT_PAIRS'}
    return game_mode in ranked_modes

# Process each JSON file
for file_path in json_files:
    with open(file_path, 'r') as file:
        data = json.load(file)
        game_mode = data.get('info', {}).get('gameMode', 'UNKNOWN')
        ranked_status = is_ranked(game_mode)
        
        # Update the counts
        game_mode_counts[game_mode] += 1
        if ranked_status:
            ranked_game_counts[game_mode] += 1

# Print the results
print("Game Mode Counts:")
for mode, count in game_mode_counts.items():
    print(f"{mode}: {count}")

print("\nRanked Game Mode Counts:")
for mode, count in ranked_game_counts.items():
    print(f"{mode}: {count}")

Game Mode Counts:

Ranked Game Mode Counts:


In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('EUW1_7075053605.parquet')

In [3]:
df

,team_id,type,principal,assists,objective
0,200,CHAMPION_KILL,8,[7],3
1,200,CHAMPION_KILL,8,[6],1
2,200,CHAMPION_KILL,8,[7],3
3,100,CHAMPION_KILL,2,[],7
4,100,ELITE_MONSTER_KILL,2,[],HORDE
...,...,...,...,...,...
56,200,CHAMPION_KILL,9,[7],5
57,100,BUILDING_KILL,9,"[7, 10]",BOT_LANE TOWER_BUILDING
58,100,BUILDING_KILL,9,"[7, 10]",BOT_LANE INHIBITOR_BUILDING
59,200,CHAMPION_KILL,7,"[9, 10]",1


In [5]:
pd.read_parquet('EUW1_7075053605_stats.parquet')

,team_id,totalGold,jungleMinionsKilled,minionsKilled,xp,level,totalDamageDone,totalDamageDoneToChampions,totalDamageTaken
1,100,4969,0,105,7045,10,36585,4118,6860
2,100,5168,94,8,5562,9,104774,4639,16759
3,100,5500,0,120,7738,11,66941,6621,12555
4,100,4266,0,92,4760,8,45935,7477,8199
5,100,4116,0,19,4279,8,11177,3473,5642
6,200,6849,8,118,8356,11,72725,7959,7859
7,200,6093,0,129,7841,11,76263,9810,8710
8,200,6689,86,10,6196,10,118018,5560,14722
9,200,6073,0,122,5780,9,47829,10564,8101
10,200,4113,0,24,5070,9,10955,2732,6457


In [11]:
import json
from pprint import pprint

with open('EUW1_7075053605.json') as f:
    metadata = json.load(f)

pprint(metadata)



{'1': ['Camille',
       'TOP',
       'uTwpW28R9vVGECWcYUBGqiauykkx9954iG1yUd3g6qDDkxFHyzDoi5y4U3pbe5ab0m1m4AC5rs8swA'],
 '10': ['Zilean',
        'UTILITY',
        'tW6bUvMWEcRKMmB_Uep2ReBR6255kHQHaJC9pE4QLTh8SN0WF9WH1CDjojlPbTxuAWLv7-VzE_Fw0Q'],
 '100_winner': False,
 '2': ['Nocturne',
       'JUNGLE',
       'PW-opgWOObI7k4ZHOUESAmcUu5kJGEUhfp90xcVq_Syp9nW_5T4b45Oqc8aLjA16UXVWBoaLzvDuqQ'],
 '3': ['Yone',
       'MIDDLE',
       '8hbFZm0f9HSj1hJp0iN0aWH3WOx2zKUhtB2Y6lxhjHG_0WrZDFhf_HhfQ8KQ3B3FS7Cqgu3Nvu0XUA'],
 '4': ['Brand',
       'BOTTOM',
       'i9BEC-CVkkzANnAXKTbyH97NSRVQGKO-vxe9XtR0J53J-EYsGrSI9NmPX09r2URQ6dQmodWKfnqXpA'],
 '5': ['Nami',
       'UTILITY',
       'mNwAMQj678BS3Hw2MuGiQP3qsMUZB274EC0MzG6AByDXBz3NkPdy5n_m1PhJuX7BUUP5eLlDzsw5Dg'],
 '6': ['Yasuo',
       'TOP',
       'VIrSCcRbr7r7vsKzAVQ55y-RwATIIGDrlx9rnrmukMuGrE8M4RxtlRP07SzfSX4iV9vjw6-Ojcjr3w'],
 '7': ['Lissandra',
       'MIDDLE',
       'gk2Mob0D0RveNCkyjPVMgVy80njIJ1oTPvULRhn1GoFPD28zg2dn_lgK4gq_hj2HxJDff